=================
# Import
=================

APIを置き換えてください．

In [ ]:
YOUR_API = "YOURAPI"

In [2]:
import os
import numpy as np
from langchain import FewShotPromptTemplate
from langchain import LLMChain, PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferMemory
from datetime import datetime
import time
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.chains import ConversationChain

gpt4 = ChatOpenAI(temperature=0.7,model_name="gpt-4-0613",openai_api_key = YOUR_API)
gpt4_low_temp = ChatOpenAI(temperature=0.5, model_name="gpt-4-0613",openai_api_key = YOUR_API)
gpt3 = ChatOpenAI(temperature=0.7,model_name="gpt-3.5-turbo",openai_api_key = YOUR_API)
os.environ["OPENAI_API_KEY"] = YOUR_API

=================
# Initialize pose
=================

In [3]:
import pyalter
import time
import numpy as np

serial_device = "/dev/tty.usbserial-FT2KYLHW"
#alter = pyalter.Alter3("serial", serial_port=serial_device)
alter =  pyalter.Alter3("simulator", simulator_address="192.168.0.49", simulator_port=11000)
#alter =  pyalter.Alter3("simulator", simulator_address="192.168.11.60", simulator_port=11000)

all_axes  = list(np.arange(1,44))
initial_value = [64,140,128,0,0,0,0,0,128,160,122,128,128,128,128,128,64,64,64,32,32,128,128,0,0,0,0,0,64,64,64,64,32,32,128,128,0,0,0,0,0,128,20]

alter.set_axes(all_axes,initial_value)
#alter.set_axes(43,185)

=================
# Create Discription
=================

In [6]:
prompt_motion_discription= """
Your task is to describe exaggerate emotional expressions and facial expressions that accompany the content of the conversation. Output motion description should be several simple motions that the android is capable of. In addition, please create a facial expression that matches the input at the beginning. The android can only move its upper body and has the same joints as a human. Output shoul be written as much detail as possible.\

Exammple1:\
'''
input : ["Driving to Hokkaido sounds like a great idea, Julia. The landscapes there could provide some great inspiration for my art."]
discription :
[
    "Showing excitement about driving to Hokkaido, several exaggerated motions are below",
    "0 Create a wide-eyed facial expression of thrill and anticipation",
    "1 Swiftly lean forward, dramatically showing interest",
    "2 Raise both hands high and spread them out widely to emphasize the idea",
    "3 Place one hand on the heart, indicating deep emotional connection to the thought",
    "4 Nod vigorously and show a broad smile, endorsing the idea emphatically",
    "5 Use both hands to make a large sweeping motion, dramatically visualizing the landscapes of Hokkaido",
    "6 Widen eyes dramatically and look around as if already absorbing the art inspiration from the landscapes",
    "7 Lean back, spread arms wide open and look upward, as if envisioning the great journey ahead",
    "8 Slowly return to rest position while maintaining a wide, satisfied smile"
]
'''

Exammple2:\
'''
input : ["drink some tea"]
discription :
[
    "drink some tea, several simple motions is below",
    "0 Create a peaceful and content facial expression, eyes slightly narrowed in a relaxed manner"
    "1 Turn towards cup",
    "2 Reach for cup",
    "3 Grasp cup",
    "4 Lift and tilt cup",
    "5 Drink",
    "6 Lower cup",
    "7 Release cup",
    "8 Return to rest"
]
'''

Guidelines:\
'''
1: Output should be list.\
2: Write as much detail as possible. Describe action step by step. Do not write any explanation.Describe exaggeration as much as possible.\
3: Android has only upper body.\
4: Create a facial expression that matches the input at the beginning
'''

input :{input}
"""

In [166]:
action_input = "pretend a snake"

In [167]:
PROMPT_motion = PromptTemplate(input_variables=["input"], template=prompt_motion_discription)
motion = LLMChain(llm=gpt3, prompt = PROMPT_motion)

output = motion.run(input= action_input)

In [168]:
output_description = output
print(output_description)

discription : 
[
    "Pretending to be a snake, several exaggerated motions are below",
    "0 Create a mischievous and playful facial expression, with a sly smile and narrowed eyes",
    "1 Arch the upper body forward, mimicking the shape of a snake ready to strike",
    "2 Extend both hands in front, with fingers pointing forward, imitating the movement of a snake's head",
    "3 Sway the upper body from side to side, in a wave-like motion, imitating the slithering movement of a snake",
    "4 Open the mouth wide, sticking out the tongue and hissing, mimicking the sound of a snake",
    "5 Curl the upper body inward, coiling up like a snake preparing to strike",
    "6 Extend the arms forward, mimicking the movement of a snake extending its fangs",
    "7 Slowly uncoil the upper body and straighten up, as if the snake is retreating",
    "8 Return to rest position with a mischievous grin and satisfied expression"
]


=================
# TOOLS
=================

In [4]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain import OpenAI, SerpAPIWrapper, LLMChain
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, OutputParserException
import re

## Agent prompt

In [170]:
prompt_template = """
Write python code to operate an android named Alter3. Here's what you need to know.\

Alter3 has 42 joints throughout its body, numbered from 1 to 42. You can move a joint by specifying its number and sending a signal. For instance, to move joints number 1,2,3, use: ''' alter.set_axes([1,2,3], [255, 100, 127]) '''. The first argument is the joint number, and the second argument is a value between 0 and 255, specifying the joint angle. Each operation takes approximately 0.1 second, so insert '''time.sleep(0.5)''' between operations.\

You have access to the following tools:
{tools}

Use the following format:

Input : the input motion you must convert to command. Input is the list, you should convert it one by one.
Thought: you should always think about what to do. You should search database at the first time by using "JsonDatabase". 
Action: the action to take, must be one of [{tool_names}].
Action Input: the input to the action.
Observation: the result of the action. 
... (this Thought/Action/Action Input/Observation should be repeat each motion)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! In the step "Thought", You can search command relative to input in database by using "JsonDatabase". The value in Json file is python code to control humanoid robot.If you have a search result, use the move as is or create a command based on it. It should not be ignored. When you now know the final answer, the previous output should be combined into a single pythoncode.

input :{input}
{agent_scratchpad}
"""

In [171]:
prompt_feedback = """
Modify python code to operate an android named Alter3. Here's what you need to know.\

Alter3 has 42 joints throughout its body, numbered from 1 to 42. You can move a joint by specifying its number and sending a signal. For instance, to move joints number 1,2,3, use: ''' alter.set_axes([1,2,3], [255, 100, 127]) '''. The first argument is the joint number, and the second argument is a value between 0 and 255, specifying the joint angle. Each operation takes approximately 0.1 second, so insert '''time.sleep(0.5)''' between operations.\

You have access to the following tools:
{tools}

Use the following format:

Input : the input motion is the python code, that you should modify with human.
Thought: You should check the python code with human. Activate the tools in the order "JsonReader", "Feedback", "CreateCommand". If Human saticefied with your code, activate "JsonWriter" tool.
Action: the action to take, must be one of [{tool_names}]. 
Action Input: the input to the action.
Observation: the result of the action. 
... (this Thought/Action/Action Input/Observation should be repeat N times.)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!: You should use tools in the order "JsonReader", "Feedback", "CreateCommand". ActionInput of "JsonReader" should be file path, it is not change. Please continue to use the file path indicated in the input. If Human saticefied with your code, activate "JsonWriter" tool. The input of "JsomMemory" should be An excerpt of the code you fixed and a short description of it.

input :{input}
{agent_scratchpad}
"""

In [5]:
import os
import json

from langchain.agents import create_json_agent, AgentExecutor
from langchain.agents.agent_toolkits import JsonToolkit
from langchain.chains import LLMChain
from langchain.llms.openai import OpenAI
from langchain.requests import TextRequestsWrapper
from langchain.tools.json.tool import JsonSpec
with open("./sample.json") as f:
    data = json.load(f)
json_spec = JsonSpec(dict_=data, max_value_length=4000)
json_toolkit = JsonToolkit(spec=json_spec)

json_agent_executor = create_json_agent(
    llm=gpt4, toolkit=json_toolkit, verbose=True
)

In [173]:
from langchain.tools.base import BaseTool
import os
from pydantic import Field

class JsonReader(BaseTool):
    """Tool that writes input to a text file."""

    name = "JsonReader"
    description = (
        "A tool for reading the JSON file"
    )
    
    # Declare file_path as a class field
    file_path: str = Field(default="")

    def __init__(self) -> None:
        super().__init__()  # Call the parent constructor
        return

    def _run(self, tool_input: str) -> str:
        tool_input = tool_input.replace("\n","")
        with open (tool_input,'r') as f:
            data = json.load(f)
        data = data.replace("```python", " ")
        data = data.replace("```", " ")
        exec(data)
        return data

    async def _arun(self, tool_input: str) -> str:
        return self._run(tool_input)

In [6]:
from langchain.tools.base import BaseTool
import os
from pydantic import Field
import json

class JsonWriter(BaseTool):
    """Tool that writes input to a text file."""

    name = "JsonWriter"
    description = (
        "A tool for writing the JSON file"
        "You should use this tool after Feedback step"
    )
    
    # Declare file_path as a class field
    file_path: str = Field(default="")

    def __init__(self) -> None:
        super().__init__()  # Call the parent constructor
        return

    def _run(self, tool_input: str) -> str:
        PROMPT_json = PromptTemplate(input_variables=["input"], template=prompt_json)
        json_chain = LLMChain(llm=gpt4_low_temp, prompt = PROMPT_json)
        JSON_format = json_chain.run(input = tool_input)
        data_to_write = {
            list(JSON_format)[0] : list(JSON_format)[1]
        }
        with open("./sample.json", 'a') as  f:
            json.dump(data_to_write,f)
        return "successfully write json file!"

    async def _arun(self, tool_input: str) -> str:
        return self._run(tool_input)

In [178]:
prompt_json = """
your task is to refinement the input.

The input is python code. You should output a description and a coherent code.

Input Example:
'''
# Set the waist and hips to lean forward quickly
alter.set_axes([13, 14], [0, 0])
time.sleep(0.1)

# Set the neck and head to lean forward quickly
alter.set_axes([10, 12], [0, 0])
time.sleep(0.1)

# Set the shoulders and arms to brace for impact
alter.set_axes([16, 17, 19, 29, 30, 32], [0, 0, 0, 0, 0, 0])
time.sleep(0.1)
'''

OUTPUT Example:
'''
["'Lean forward and fighting stance'", "#Lean forward and fighting stance\n alter.set_axes([13, 14,10, 12,16, 17, 19, 29, 30, 32], [0, 0, 0, 0,0, 0, 0, 0, 0, 0])]
'''

Guideline:
    1: Output format should be list, ["'short description of cede'", "#short description of cede\n pyalter([],[])"]
    2: Element with index 0 of output list should be the short description of input motion. Enclose with "' and '".
    3: Element with index 0 of output list should be the one sentence code with description by #.

input :{input}
"""

In [179]:
create_command_temp = """
Write python code to operate an android named Alter3. Here's what you need to know.\

Alter3 has 42 joints throughout its body, numbered from 1 to 42. You can move a joint by specifying its number and sending a signal. For instance, to move joints number 1,2,3, use: ''' alter.set_axes([1,2,3], [255, 100, 127]) '''. The first argument is the joint number, and the second argument is a value between 0 and 255, specifying the joint angle. Each operation takes approximately 0.1 second, so insert '''time.sleep(0.1)''' between operations.\

'''
Alter3's Joints:
- Axis 1: Eyebrows. 255 = angry, 0 = surprised, 64 = neutral.
- Axis 2: Pupils (horizontal). 255 = left, 0 = right, 140 = neutral.
- Axis 3: Pupils (vertical). 255 = up, 0 = down, 128 = neutral.
- Axis 4: Eyes. 255 = closed, 0 = open.
- Axis 5-6: Left/Right cheek. 255 = raised (smile), 0 = lowered.
- Axis 7: Lips. 255 = puckered, 0 = relaxed.
- Axis 8: Mouth. 255 = open, 0 = closed.
- Axis 9: Head tilt. 255 = left, 0 = right, 128 = neutral.
- Axis 10: Head up/down. 255 = down, 0 = up, 160 = neutral.
- Axis 11: Head rotate. 255 = left, 0 = right, 122 = neutral.
- Axis 12: Neck nod. 255 = backward, 0 = forward, 128 = neutral.
- Axis 13: Hips tilt. 255 = left, 0 = right, 128 = neutral.
- Axis 14: Waist bend. 255 = backward, 0 = forward, 128 = neutral.
- Axis 15: Abdomen rotation. 255 = left, 0 = right, 128 = neutral.
- Axis 16: Left shoulder up/down. 255 = up, 0 = down, 128 = neutral.
- Axis 17: Left shoulder forward/back. 255 = forward, 0 = back, 64 = neutral.
- Axis 18: Left armpit open/close. 255 = open, 0 = close, 64 = neutral.
- Axis 19: Left arm lift. 255 = up, 0 = down, 64 = neutral.
- Axis 20: Left upper arm rotation. 255 = left, 0 = right, 32 = neutral.
- Axis 21: Left elbow bend. 255 = bent, 0 = straight, 32 = neutral.
- Axis 22: Left forearm twist. 255 = outward, 0 = inward, 128 = neutral.
- Axis 23: Left wrist bend. 255 = straight, 0 = bent, 128 = neutral.
- Axis 24: Left wrist side bend. 255 = left, 0 = right.
- Axis 25: Left thumb. 255 = bent, 0 = straight.
- Axis 26: Left index finger. 255 = bent, 0 = straight.
- Axis 27: Left middle finger. 255 = bent, 0 = straight.
- Axis 28: Left ring/little fingers. 255 = bent, 0 = straight.
- Axis 29: Right shoulder up/down. 255 = up, 0 = down, 128 = neutral.
- Axis 30: Right shoulder forward/back. 255 = forward, 0 = back, 64 = neutral.
- Axis 31: Right armpit open/close. 255 = open, 0 = close, 64 = neutral.
- Axis 32: Right arm lift. 255 = up, 0 = down, 64 = neutral.
- Axis 33: Right upper arm rotation. 255 = right, 0 = left, 32 = neutral.
- Axis 34: Right elbow bend. 255 = bent, 0 = straight, 32 = neutral.
- Axis 35: Right forearm twist. 255 = outward, 0 = inward, 128 = neutral.
- Axis 36: Right wrist bend. 255 = straight, 0 = bent, 128 = neutral.
- Axis 37: Right wrist side bend. 255 = right, 0 = left.
- Axis 38: Right thumb. 255 = bent, 0 = straight.
- Axis 39: Right index finger. 255 = bent, 0 = straight.
- Axis 40: Right middle finger. 255 = bent, 0 = straight.
- Axis 41: Right ring/little fingers. 255 = bent, 0 = straight.
- Axis 42: Whole body raise/lower. 255 = raised, 0 = lowered, 128 = neutral.
'''

OUTPUT Example1:drink some tea\
'''
# Face the front with the cups
alter.set_axes([13, 11, 9], [150, 150, 255])
time.sleep(0.1)
# Reaching for a virtual tea cup
alter.set_axes([29, 32, 30, 31], [200, 150, 150, 200])
time.sleep(0.1)
# Put your hand on the virtual cup
alter.set_axes([36, 37], [0, 0])
time.sleep(0.1)
# Hold the virtual cup with fingers
alter.set_axes([38, 39, 40, 41], [255, 255, 255, 255])
time.sleep(0.1)
# Bend the elbows and bring the teacup to your mouth.
alter.set_axes([34], [255])
time.sleep(0.1)
# Move your wrist and put the teacup to your mouth
alter.set_axes([35,37], [105,0])
time.sleep(0.1)
# Open your mouth and drink tea
alter.set_axes([8,12], [200, 240])
time.sleep(0.1)
# Stretch out your elbows and bring back the tea cup to the desk.
alter.set_axes([34], [0])
time.sleep(0.1)
# Release tea cup
alter.set_axes([36, 37], [255, 255])
time.sleep(0.1)
'''

Guidelines:\
1: Output should be only python code. Do not insert any syntax highlighting like ```.
2: Do not Create an instance of alter3.
3: Do not insert python syntax highlighting like ```python ```.
4: Do not write "import alter".
5: Use # and write short description of code.

input:{input}
"""

In [180]:
from langchain.tools.base import BaseTool
import os
from pydantic import Field

class CreateCommand(BaseTool):
    """Tool that writes input to a text file."""

    name = "CreateCommand"
    description = (
        "A tool for creating motion command"
        "Input should be the description of motion in one sentense"
    )
    
    # Declare file_path as a class field
    file_path: str = Field(default="")

    def __init__(self) -> None:
        super().__init__()  # Call the parent constructor
        return

    def _run(self, tool_input: str) -> str:
        PROMPT_code = PromptTemplate(input_variables=["input"], template=create_command_temp)
        code = LLMChain(llm=gpt4_low_temp, prompt = PROMPT_code)
        created_command = code.run(input = tool_input)
        return created_command

    async def _arun(self, tool_input: str) -> str:
        return self._run(tool_input)

In [181]:
from langchain.tools.base import BaseTool
import os
from pydantic import Field
import time

class Feedback(BaseTool):
    """Tool that writes input to a text file."""

    name = "Feedback"
    description = (
        "Tools for human checking and feedback of output code"
        "If you have generated code, run this tool to correct it"
    )
    
    # Declare file_path as a class field
    file_path: str = Field(default="")

    def __init__(self) -> None:
        super().__init__()  # Call the parent constructor
        return

    def _run(self, tool_input: str) -> str:
        tool_input = tool_input.replace("'''", "")
        exec(tool_input)
        time.sleep(1)
        user_input = input("Should I change code?: ")
        return user_input

    async def _arun(self, tool_input: str) -> str:
        return self._run(tool_input)

In [182]:
tools = [
    Tool(
        name = "JsonDatabase",
        func=json_agent_executor.run,
        description="A database of motions and corresponded python commands for humanoid robot. "
    ),
    CreateCommand()
]

In [183]:
tools_feedback = [
    CreateCommand(),
    JsonWriter(),
    JsonReader(),
    Feedback()
]

In [184]:
# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[BaseTool]

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

In [185]:
prompt = CustomPromptTemplate(
    template=prompt_template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

In [186]:
PromprFeedback = CustomPromptTemplate(
    template=prompt_feedback,
    tools=tools_feedback,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

In [187]:
class CustomOutputParser(AgentOutputParser):

    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise OutputParserException(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)
    
output_parser = CustomOutputParser()

In [188]:
llm_chain = LLMChain(llm=gpt4_low_temp, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    stop=["\nObservation:"],
    output_parser=output_parser,
    allowed_tools=tool_names,
    verbose=True
)

In [189]:
llm_chain_feedback = LLMChain(llm=gpt4_low_temp, prompt=PromprFeedback)
tool_names_feedback = [tool.name for tool in tools_feedback]
agent_feedback = LLMSingleActionAgent(
    llm_chain=llm_chain_feedback,
    stop=["\nObservation:"],
    output_parser=output_parser,
    allowed_tools=tool_names_feedback,
    verbose=True
)

=================
# Create Code
=================

In [190]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True,max_execution_time=2400, max_iterations = 50 )

In [191]:
description_input = output_description

In [192]:
output = agent_executor.run(description_input)



> Entering new AgentExecutor chain...
Thought: I need to convert the first motion description to a command. I will check the JsonDatabase to see if there is a corresponding command for "Pretending to be a snake, several exaggerated motions are below".
Action: JsonDatabase
Action Input: "Pretending to be a snake, several exaggerated motions are below"

> Entering new AgentExecutor chain...
Action: json_spec_list_keys
Action Input: data
Observation: ["'Bend left elbow and raise left hand to head height.'", "'Bend right elbow and raise right hand to head height.'", "'Spread both hands out in front of body.'", "'smile", "'Put your left hand on your chest.'", "'Put your right hand on your chest.'", "'Guitar position'", "'# Eyebrows to furrowed'", "'# Set pupils to neutral position'", "'# Open eyes wide'", "'# Tilt the hips forward'", "'# Bend the waist forward'", "'# Tilt the head forward'", "'# Bend the neck forward'", "'# Raise the left and right shoulders'", "'# Bend the left and right

=================
# Feedback
=================

In [ ]:
with open("./motion_command.json","w") as f:
    json.dump(output)

In [207]:
def parse_code(code_str):
    lines = code_str.split('\n')
    
    parsed_dict = {}
    current_comment = None
    
    for line in lines:
        line = line.strip()
        
        # Skip empty lines
        if not line:
            continue
        
        # If the line starts with a comment
        if line.startswith("#"):
            current_comment = line
        else:
            # If there's a valid comment preceding the code line
            if current_comment:
                parsed_dict["'" + current_comment + "'"] = current_comment + "\n" + line
                current_comment = None  # Reset for next set
            
    return parsed_dict

In [208]:
with open("./motion_command.json") as f:
    code =  json.load(f)

code_blocks = code.split("\n\n")

for i in range(len(code_blocks)):
    with open(f"./output/{i}.json", "w") as f:
        json_data = json.dump(code_blocks[i],f, indent=4)

# Print or save the JSON data as needed
print(json_data)

None


In [210]:
path = "./output"
all_names = os.listdir(path)
all_names.sort(key=lambda x: int(x.split('.')[0]))
counter = 0
while True:
    initial_value = [64,140,128,0,0,0,0,0,128,160,122,128,128,128,128,128,64,64,64,32,32,128,128,0,0,0,0,0,64,64,64,64,32,32,128,128,0,0,0,0,0,128,185]
    alter.set_axes(all_axes,initial_value)
    # Write JSON
    all_names = os.listdir(path)
    all_names.sort(key=lambda x: int(x.split('.')[0]))
    #EXEC DATA
    for i in range(len(all_names)):
        with open (f"{path}/{all_names[i]}",'r') as f:
            data = json.load(f)
        data = "#" + data
        data = data.replace("```python", " ")
        data = data.replace("```", " ")
        print(all_names[i])
        exec(data)

    #GET FEEDBACK
    user_input = input("Should I change code?: ")
    if user_input == "retry":
        exec(data)
        user_input = input("Should I change code?: ")
    if user_input == "perfect":
        parsed_dict = parse_code(data)
        with open("./sample.json",'a') as f:
            json.dump(parsed_dict, f)
        break

    first_char = user_input[0] if user_input else None
    with open(f"{path}/{first_char}.json","r") as f:
        extract_code = json.load(f)

    # CREATE CODE
    PROMPT_modify = PromptTemplate(input_variables=["input"], template=prompt_modify)
    modify = LLMChain(llm=gpt4_low_temp, prompt = PROMPT_modify, verbose=True)

    modify_input = f"user input:{user_input},\n code:{extract_code}"

    output = modify.run(modify_input)
    with open(f"{path}/{first_char}.json", 'w') as f:
        json.dump(output, f)
    counter += 1

0.json
1.json
2.json
3.json
4.json
5.json
6.json
7.json
8.json
9.json
10.json


/Users/tnoinkwms/miniforge3/envs/alter/lib/python3.8/site-packages/pyalter/alter3.py:79: UserWarning: invalid arguments.
  warnings.warn("invalid arguments.")




> Entering new LLMChain chain...
Prompt after formatting:

Your task is modify the inputed code with human instruction. If human says "create", you should create the motion as instruction. The code is the motion command of Android Alter3.
Here is the information you should know.
Alter3 has 42 joints throughout its body, numbered from 1 to 42. You can move a joint by specifying its number and sending a signal. For instance, to move joints number 1,2,3, use: ''' alter.set_axes([1,2,3], [255, 100, 127]) '''. The first argument is the joint number, and the second argument is a value between 0 and 255, specifying the joint angle. Each operation takes approximately 0.1 second, so insert '''time.sleep(0.1)''' between operations.
'''
Alter3's Joints:
- Axis 1: Eyebrows. 255 = angry, 0 = surprised, 64 = neutral.
- Axis 2: Pupils (horizontal). 255 = left, 0 = right, 140 = neutral.
- Axis 3: Pupils (vertical). 255 = up, 0 = down, 128 = neutral.
- Axis 4: Eyes. 255 = closed, 0 = open.
- Axis 5-6

/Users/tnoinkwms/miniforge3/envs/alter/lib/python3.8/site-packages/pyalter/alter3.py:79: UserWarning: invalid arguments.
  warnings.warn("invalid arguments.")




> Entering new LLMChain chain...
Prompt after formatting:

Your task is modify the inputed code with human instruction. If human says "create", you should create the motion as instruction. The code is the motion command of Android Alter3.
Here is the information you should know.
Alter3 has 42 joints throughout its body, numbered from 1 to 42. You can move a joint by specifying its number and sending a signal. For instance, to move joints number 1,2,3, use: ''' alter.set_axes([1,2,3], [255, 100, 127]) '''. The first argument is the joint number, and the second argument is a value between 0 and 255, specifying the joint angle. Each operation takes approximately 0.1 second, so insert '''time.sleep(0.1)''' between operations.
'''
Alter3's Joints:
- Axis 1: Eyebrows. 255 = angry, 0 = surprised, 64 = neutral.
- Axis 2: Pupils (horizontal). 255 = left, 0 = right, 140 = neutral.
- Axis 3: Pupils (vertical). 255 = up, 0 = down, 128 = neutral.
- Axis 4: Eyes. 255 = closed, 0 = open.
- Axis 5-6

...

/Users/tnoinkwms/miniforge3/envs/alter/lib/python3.8/site-packages/pyalter/alter3.py:79: UserWarning: invalid arguments.
  warnings.warn("invalid arguments.")




> Entering new LLMChain chain...
Prompt after formatting:

Your task is modify the inputed code with human instruction. If human says "create", you should create the motion as instruction. The code is the motion command of Android Alter3.
Here is the information you should know.
Alter3 has 42 joints throughout its body, numbered from 1 to 42. You can move a joint by specifying its number and sending a signal. For instance, to move joints number 1,2,3, use: ''' alter.set_axes([1,2,3], [255, 100, 127]) '''. The first argument is the joint number, and the second argument is a value between 0 and 255, specifying the joint angle. Each operation takes approximately 0.1 second, so insert '''time.sleep(0.1)''' between operations.
'''
Alter3's Joints:
- Axis 1: Eyebrows. 255 = angry, 0 = surprised, 64 = neutral.
- Axis 2: Pupils (horizontal). 255 = left, 0 = right, 140 = neutral.
- Axis 3: Pupils (vertical). 255 = up, 0 = down, 128 = neutral.
- Axis 4: Eyes. 255 = closed, 0 = open.
- Axis 5-6

/Users/tnoinkwms/miniforge3/envs/alter/lib/python3.8/site-packages/pyalter/alter3.py:79: UserWarning: invalid arguments.
  warnings.warn("invalid arguments.")




> Entering new LLMChain chain...
Prompt after formatting:

Your task is modify the inputed code with human instruction. If human says "create", you should create the motion as instruction. The code is the motion command of Android Alter3.
Here is the information you should know.
Alter3 has 42 joints throughout its body, numbered from 1 to 42. You can move a joint by specifying its number and sending a signal. For instance, to move joints number 1,2,3, use: ''' alter.set_axes([1,2,3], [255, 100, 127]) '''. The first argument is the joint number, and the second argument is a value between 0 and 255, specifying the joint angle. Each operation takes approximately 0.1 second, so insert '''time.sleep(0.1)''' between operations.
'''
Alter3's Joints:
- Axis 1: Eyebrows. 255 = angry, 0 = surprised, 64 = neutral.
- Axis 2: Pupils (horizontal). 255 = left, 0 = right, 140 = neutral.
- Axis 3: Pupils (vertical). 255 = up, 0 = down, 128 = neutral.
- Axis 4: Eyes. 255 = closed, 0 = open.
- Axis 5-6

/Users/tnoinkwms/miniforge3/envs/alter/lib/python3.8/site-packages/pyalter/alter3.py:79: UserWarning: invalid arguments.
  warnings.warn("invalid arguments.")


In [216]:
for i in range(len(all_names)):
    with open (f"{path}/{all_names[i]}",'r') as f:
        data = json.load(f)
    data = "#" + data
    data = data.replace("```python", " ")
    data = data.replace("```", " ")
    print(all_names[i])
    exec(data)
    time.sleep(0.1)

0.json
1.json
2.json
3.json
4.json
5.json
6.json
7.json
8.json
9.json
10.json
